# Solr Client

In [1]:
from ltr.client import SolrClient
client = SolrClient()

# Download & Build Index (run once)

If you don't already have the downloaded dependencies; if you don't have TheMovieDB data indexed run this

In [2]:
from ltr import download

corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'
download([corpus, judgments], dest='data/');

data/tmdb.json already exists
data/title_judgments.txt already exists


In [4]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

Reconfig from disk...
Deleted index tmdb [Status: 400]
{
  "responseHeader":{
    "status":400,
    "QTime":71},
  "error":{
    "metadata":[
      "error-class","org.apache.solr.common.SolrException",
      "root-error-class","org.apache.solr.common.SolrException"],
    "msg":"Cannot unload non-existent core [tmdb]",
    "code":400}}

Created index tmdb [Status: 200]
Reindexing...
Indexed 0 movies (last Black Mirror: White Christmas)
Indexed 100 movies (last Apocalypse Now)
Indexed 200 movies (last Crooks in Clover)
Indexed 300 movies (last For a Few Dollars More)
Indexed 400 movies (last Downfall)
Indexed 500 movies (last Finding Nemo)
Indexed 600 movies (last Platoon)
Indexed 700 movies (last Night of the Living Dead)
Indexed 800 movies (last Evangelion: 1.0: You Are (Not) Alone)
Indexed 900 movies (last Batman: Assault on Arkham)
Indexed 1000 movies (last Riley's First Date?)
Indexed 1100 movies (last The Raid)
Indexed 1200 movies (last Falling Down)
Indexed 1300 movies (last Kal H

Done [Status: 200]
Indexed 20000 movies (last Left Behind III: World at War)
Indexed 20100 movies (last Dragon Ball Z: Lord Slug)
Indexed 20200 movies (last The Adventures of Sherlock Holmes)
Indexed 20300 movies (last Billy's Hollywood Screen Kiss)
Indexed 20400 movies (last Short Night of Glass Dolls)
Indexed 20500 movies (last Kawa)
Indexed 20600 movies (last Bears)
Indexed 20700 movies (last Pyrates)
Indexed 20800 movies (last Bastard Out of Carolina)
Indexed 20900 movies (last The Mole People)
Indexed 21000 movies (last Till Human Voices Wake Us)
Indexed 21100 movies (last It's a Wonderful Afterlife)
Indexed 21200 movies (last The Bingo Long Traveling All-Stars & Motor Kings)
Indexed 21300 movies (last Ciao! Manhattan)
Indexed 21400 movies (last The Night They Raided Minsky's)
Indexed 21500 movies (last The Girl Can't Help It)
Indexed 21600 movies (last Sam Peckinpah's West: Legacy of a Hollywood Renegade)
Indexed 21700 movies (last A Guy Named Joe)
Indexed 21800 movies (last Odd 

## Features for movie titles

We'll be searching movie titles (think searching for a specific movie on Netflix). And we have a set of judgments around the appropriatte movie to return. IE search for "Star Wars" return good star wars matches, in quality order...

These cover various aspects of the problem (searching title by phrase, title bm25 score, release date, etc). We'll use this to explore and analyze a simple model

In [5]:
client.reset_ltr(index='tmdb')

config = [
    #1
    {
      "name" : "title_has_phrase",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:\"${keywords})\"^=1"
      }
    },
    #2
    {
      "name" : "title_has_terms",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})^=1"
      }
    },
    #3
    {
      "name" : "title_bm25",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})"
      }
    },
    #4
    {
      "name" : "overview_bm25",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    #5
    {
      "name" : "overview_phrase_bm25",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:\"${keywords}\""
      }
    },
    #6
    {
      "name" : "title_fuzzy",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!lucene df=title}${keywords}~"
      }
    },
    #7
    {
      "name" : "release_year",
      "store": "title",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}def(release_year,2000)"
      }
    }

]

client.create_featureset(index='tmdb', name='title', ftr_config=config)

Created title feature store under tmdb: [Status: 200]


## Training Set Generation

Log out features for each of the above queries out to a training set file

In [ ]:
from ltr.judgments import judgments_open
from ltr.log import FeatureLogger
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='title')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(qid=qid, 
                               judgments=query_judgments,
                               keywords=judgment_list.keywords(qid))


Recognizing 40 queries...
> /Users/doug/ws/hello-ltr/ltr/judgments.py(31)keywords()
-> return self.kw_with_weight[qid][0]
(Pdb) c
Searching tmdb [Status: 200]
Discarded 0 Keep 41
> /Users/doug/ws/hello-ltr/ltr/judgments.py(31)keywords()
-> return self.kw_with_weight[qid][0]


## Feature Search: which features work best?

What combination of these features work best? Train a model with every combination, and use k-fold cross valudation (see `kcv=15` below). The combination with the best NDCG is output

In [ ]:
from ltr.ranklib import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               training_set=training_set,
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,7],
                                               featureSet='title')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

## Compare to model w/ all features

Compare the features output above (something like...)

```
Impact of each feature on the model
7 - 17618.35445148437
4 - 16165.586045512271
3 - 10958.610341321868
5 - 9256.821192289186
1 - 1436.0640878600943
```

to one trained with the full model. Notice how features have different impacts. This is due to feature dependency

In [ ]:
from ltr.ranklib import train
trainLog  = train(client,
                  training_set=training_set,
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  features=[1,2,3,4,5,6,7],
                  featureSet='title',
                  index='tmdb',
                  modelName='title')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

## Bias towards fewer features

By adding a 'cost', to feature search, we add a multiplier that punishes models with more features slightly. This results in a tiny bias towards simpler models all things being equal. As we'd prefer one that doesn't need to execute more features

In [ ]:
from ltr.ranklib import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               training_set=training_set,
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               featureCost=0.1,# adjustedNDCG = NDCG * ( (1.0-cost) ^ num_features)
                                               features=[1,2,3,4,5,6,7],
                                               featureSet='title')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

# Evaluating the Model

It's interesting to see what features our model makes use of, but we need guidance on adding additional features to the model. We know our model is an ensemble of decision trees. Wouldn't it be cool if we could trace where documents end up on that decision tree?

Specifically, we care about problems. Or what we will call affectionately *whoopsies*. 

As a 'whoopsie' example, consider the query "Rambo". if a '0' document like 'First Daughter' ranked the same or higher than a '4' document ("Rambo")., that's a problem. It's also an opportunity for improvement. We'd want to isolate that, see if it's indicative of a broader trend, and thus worth adding a feature for.

Let's see a concrete example

In [ ]:
from ltr.MART_model import eval_model
from itertools import groupby

features, _ = client.feature_set(index='tmdb', name='title')

for qid, query_judgments in groupby(training_set, key=lambda j: j.qid):
    print(qid)
    if qid == 1: # Rambo
        model = eval_model(modelName='title',
                               features=features,
                               judgments=query_judgments)

        print()
        print("## Evaluating graded docs for search keywords 'Rambo'")
        print()
        print(model)
        break

## Examining our evaluation for whoopsies

Let's looks at one tree in our ensemble, te see how it was evaluated.

```
if title_bm25 > 10.664251:
  if title_phrase > 0.0:
    if title_bm25 > 13.815164:
      if release_year > 2000.0:
        <= 0.1215(0/0/)
      else:
        <= 0.1240(0/0/)
    else:
      if title_bm25 > 10.667803:
        if overview_bm25 > 0.0:
          <= 0.1194(0/0/)
        else:
          <= 0.1161(1/0/)
      else:
        <= 0.1264(0/0/)
  else:
    <= 0.0800(0/0/)
else:
  if title_phrase > 0.0:
    if title_bm25 > 8.115499:
      if title_bm25 > 8.217656:
        <= 0.1097(2/1/qid:40:2(12180)-3(140607))
      else:
        <= 0.1559(0/0/)
    else:
      <= -0.0021(2/1/qid:40:2(1895)-3(330459))
  else:
    <= -0.1093(25/1/qid:40:0(85783)-3(1892))
```

You'll notice here this tree is represented by a series of if statements, where the feature's name is used. This is handy as it lets us take apart the structure of the tree.

You'll also notice the leaf nodes starting with 

```
<=
```

These leaf nodes have a floating point value, corresponding to the relevance score that documents ending up here will have. Each leaf also has three items in paranthesis, such as `(2/1/qid:40:2(1895)-3(330459))`. This is a report summarizing the result of evaluating the tree on the provided judgment list. Indicating:


```



   +--- 2 Documents evaluated to this leaf node                   +-- max grade doc eval'd to this leaf
   |                                                              |
   | +----- 1 'whoopsie' occured                                  |  +-- corresp. doc id of max doc
   | |                                                            |  |
   | |   +--- details on each whoopsie ----------- qid:40:2(1985)-3(330459)
   | |   |                                              | |  |
  (2/1/qid:40:2(1895)-3(330459))                        | |  |
                                                        | |  + doc id of min graded doc
                                                        | |
                                                        | + min grade of docs eval'd to this leaf
                                                        |
                                                        + query id of whoopsie from judgments
```


Looking at Star Wars, our biggest issues in this tree are with the bottom-most leaf. Here

```
if title_bm25 > 10.664251:
  ...
else:
  if title_phrase > 0.0:
    ...
  else:
    <= -0.1093(25/1/qid:40:0(85783)-3(1892))
```


Document 85783 (a '0') and doc 1892 are given the same grade.

### Whoopsie, from the query perspective

Whoopsies can also be examined at the "query" level to see for a query id, how many whoopsies existed, and what was the evaluation for that query at each tree. This can help see if an error was fixed later in the ensemble of trees.

In [ ]:
whoopsies = model.whoopsies()
for qid, whoopsie in whoopsies.items():
    print("== QID %s ==" % qid)
    print("%s - %s" % (whoopsie.count, whoopsie.totalMagnitude))
    print(whoopsie.perTreeReport())

## Examine problem doc 319074

(notice nothing mentions 'star wars')

In [ ]:


client.get_doc(index='tmdb', doc_id=1368)

## Add a feature: collection name

We have an intuition about our data, there is a field for the movies "collection name". See it here below:

In [ ]:
from ltr.helpers.movies import get_movie
get_movie(1368)

## Now reindex with collection name...

We'll add collection name, and reindex.

In [ ]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

def add_collection_and_char_name(src_movie, base_doc):
    if 'belongs_to_collection' in src_movie and src_movie['belongs_to_collection'] is not None:
        if 'name' in src_movie['belongs_to_collection']:
            base_doc['collection_name_en'] = src_movie['belongs_to_collection']['name']
            
    if 'cast' in src_movie and src_movie['cast'] is not None:
        characters = [cast['character'] for cast in get_movie(1368)['cast']][:5]
        base_doc['top_cast_en'] = characters
    return base_doc

movies=indexable_movies(movies='data/tmdb.json', enrich=add_collection_and_char_name)
rebuild(client, index='tmdb', doc_src=movies)

Confirm it's in our doc now...

In [ ]:
client.get_doc(index='tmdb', doc_id=319074)

## Add it to the features, and regenerate training data....

In [ ]:
config = [
    #1
    {
      "name" : "title_has_phrase",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:\"${keywords})\"^=1"
      }
    },
    #2
    {
      "name" : "title_has_terms",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})^=1"
      }
    },
    #3
    {
      "name" : "title_bm25",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})"
      }
    },
    #4
    {
      "name" : "overview_bm25",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    #5
    {
      "name" : "overview_phrase_bm25",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:\"${keywords}\""
      }
    },
    #6
    {
      "name" : "title_fuzzy",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!lucene df=title}${keywords}~"
      }
    },
    #7
    {
      "name" : "release_year",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}def(release_year,2000)"
      }
    },
    #8 Collection Name BM25 Score
    {
      "name" : "coll_name_bm25",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "collection_name_en:(${keywords})"
      }
    },
    #9 Collection Name Phrase BM25 Score
    {
      "name" : "coll_name_phrase_bm25",
      "store": "title2",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "collection_name_en:\"${keywords}\""
      }
    }

]


client.create_featureset(index='tmdb', name='title2', ftr_config=config)

from ltr.judgments import judgments_open
from ltr.log import log_query
from itertools import groupby

training_set=[]
with judgments_open('data/title_judgments.txt') as judg_list:
    # For each query's judgments log features, and add to our training set...
    for qid, query_judgments in groupby(judg_list, key=lambda j: j.qid):
        query_training_set, discarded = log_query(client, index='tmdb', 
                                                  judgments=query_judgments, feature_set='title2')
        training_set.extend(query_training_set)
        print("Got %s Training Samples" % len(training_set))

## Now a feature search

And do a feature search over these new features (go get some coffee).

We also up the number of trees & leafs to see if it has an impact

In [ ]:
from ltr.ranklib import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               training_set=training_set,
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,3,4,5,6,7,8,9],
                                               featureSet='title2')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

## Review new feature impacts

Impact of each feature on the model... this is the best mix. Feature 8 helps, but not feature 9 as much. Interesting

```
4 - 18032.527656827504
3 - 9801.409052757816
5 - 8051.741259194476
7 - 5711.964176322393
8 - 3798.6132329430748
1 - 1439.2180228991883
```

## Now save away this model

In [ ]:
from ltr.ranklib import train
trainLog  = train(client,
                  training_set=training_set,
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  features=[1,3,4,5,7,8],
                  featureSet='title2',
                  index='tmdb',
                  modelName='title2')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

In [ ]:
from ltr import search
search(client, "rambo", modelName='title2')

## Examine Model 2

In [ ]:
from ltr.MART_model import eval_model
from itertools import groupby

features, _ = client.feature_set(index='tmdb', name='title2')

for qid, query_judgments in groupby(training_set, key=lambda j: j.qid):
    if qid == 1:
        model = eval_model(modelName='title',
                           features=features,
                           judgments=query_judgments)

        print()
        print("## Evaluating graded docs for search keywords 'Rambo'")
        print()
        print(model)
        break

In [ ]:
whoopsies = model.whoopsies()
for qid, whoopsie in whoopsies.items():
    print("== QID %s ==" % qid)
    print("%s - %s" % (whoopsie.count, whoopsie.totalMagnitude))
    print(whoopsie.perTreeReport())

```
== QID 1 ==
10 - 40
tree:0=>0(319074)-4(1368);tree:1=>0(319074)-4(1368);tree:2=>0(319074)-4(1368);tree:3=>0(319074)-4(1368);tree:4=>0(319074)-4(1368);tree:5=>0(319074)-4(1368);tree:6=>0(319074)-4(1368);tree:7=>0(319074)-4(1368);tree:8=>0(319074)-4(1368);tree:9=>0(319074)-4(1368)
```

In [ ]:
from ltr.helpers.movies import get_movie
[cast['character'] for cast in get_movie(1368)['cast']][:5]

In [ ]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

def add_collection_and_char_name(src_movie, base_doc):
    if 'belongs_to_collection' in src_movie and src_movie['belongs_to_collection'] is not None:
        if 'name' in src_movie['belongs_to_collection']:
            base_doc['collection_name_en'] = src_movie['belongs_to_collection']['name']
            
    if 'cast' in src_movie and src_movie['cast'] is not None:
        characters = [cast['character'] for cast in get_movie(1368)['cast']][:5]
        base_doc['top_chars_en'] = characters
    return base_doc

movies=indexable_movies(movies='data/tmdb.json', enrich=add_collection_and_char_name)
rebuild(client, index='tmdb', doc_src=movies)

In [ ]:
client.get_doc(index='tmdb', doc_id=1892)